In [ ]:
# Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt

import requests

pd.options.display.max_columns = None

## OpenSea Events: LAND

I used [OpenSea API](https://docs.opensea.io/reference/api-overview) to retrieve all the historical events (e.g., auctions, sales, transfers) related to LANDs from 2018 to 2020.

In [2]:
events_land = pd.read_csv("./data/events_land_181920.csv")
events_land

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,7,11,14,19,20,22,34,94,98,99,102,105,108,114,119,122,123,124,125,132,133,134,135,137,138,139,140,141,145,146,150,151,152,153,157,159,160,161,162,164,166,167,169,171) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,approved_account,asset_bundle,auction_type,bid_amount,collection_slug,contract_address,created_date,custom_event_name,dev_fee_payment_event,dev_seller_fee_basis_points,duration,ending_price,event_type,id,is_private,owner_account,payment_token,quantity,seller,starting_price,total_price,winner_account,listing_time,asset.id,asset.token_id,asset.num_sales,asset.background_color,asset.image_url,asset.image_preview_url,asset.image_thumbnail_url,asset.image_original_url,asset.animation_url,asset.animation_original_url,asset.name,asset.description,asset.external_link,asset.asset_contract.address,asset.asset_contract.asset_contract_type,asset.asset_contract.created_date,asset.asset_contract.name,asset.asset_contract.nft_version,asset.asset_contract.opensea_version,asset.asset_contract.owner,asset.asset_contract.schema_name,asset.asset_contract.symbol,asset.asset_contract.total_supply,asset.asset_contract.description,asset.asset_contract.external_link,asset.asset_contract.image_url,asset.asset_contract.default_to_fiat,asset.asset_contract.dev_buyer_fee_basis_points,asset.asset_contract.dev_seller_fee_basis_points,asset.asset_contract.only_proxied_transfers,asset.asset_contract.opensea_buyer_fee_basis_points,asset.asset_contract.opensea_seller_fee_basis_points,asset.asset_contract.buyer_fee_basis_points,asset.asset_contract.seller_fee_basis_points,asset.asset_contract.payout_address,asset.permalink,asset.collection.banner_image_url,asset.collection.chat_url,asset.collection.created_date,asset.collection.default_to_fiat,asset.collection.description,asset.collection.dev_buyer_fee_basis_points,asset.collection.dev_seller_fee_basis_points,asset.collection.discord_url,asset.collection.display_data.card_display_style,asset.collection.external_url,asset.collection.featured,asset.collection.featured_image_url,asset.collection.hidden,asset.collection.safelist_request_status,asset.collection.image_url,asset.collection.is_subject_to_whitelist,asset.collection.large_image_url,asset.collection.medium_username,asset.collection.name,asset.collection.only_proxied_transfers,asset.collection.opensea_buyer_fee_basis_points,asset.collection.opensea_seller_fee_basis_points,asset.collection.payout_address,asset.collection.require_email,asset.collection.short_description,asset.collection.slug,asset.collection.telegram_url,asset.collection.twitter_username,asset.collection.instagram_username,asset.collection.wiki_url,asset.decimals,asset.token_metadata,asset.owner.user,asset.owner.profile_img_url,asset.owner.address,asset.owner.config,from_account.user.username,from_account.profile_img_url,from_account.address,from_account.config,to_account.user.username,to_account.profile_img_url,to_account.address,to_account.config,transaction.block_hash,transaction.block_number,transaction.from_account.user.username,transaction.from_account.profile_img_url,transaction.from_account.address,transaction.from_account.config,transaction.id,transaction.timestamp,transaction.to_account.user,transaction.to_account.profile_img_url,transaction.to_account.address,transaction.to_account.config,transaction.transaction_hash,transaction.transaction_index,to_account.user,to_account,asset.owner.user.username,from_account.user,payment_token.id,payment_token.symbol,payment_token.address,payment_token.image_url,payment_token.name,payment_token.decimals,payment_token.eth_price,payment_token.usd_price,transaction.from_account.user,transaction,from_account,seller.user.username,seller.profile_img_url,seller.address,seller.config,winner_account.user,winner_account.profile_img_url,winner_account.address,winner_account.config,winner_account.user.username,transaction.to_account.user.username,seller.user,dev_fee_payment_event.asset,dev_fee_payment_event.asset_bundle,dev_fee_payment_event.event_type,dev_fee_payment_event.event_timestamp,dev_fee_payment_event.auction_type,dev_fee_payment_event.total_price,dev_fee_payment_event.payment_token.id,dev_fee_payment_event.payment_token.symbol,dev_fee_payment_ev

In [ ]:
url_decentraland = "https://api.decentraland.org/"
response = requests.get(url_decentraland + "v2/tiles")
tiles = pd.DataFrame.from_dict(response.json()["data"], orient="index").reset_index(drop=True)
tiles = tiles.rename(columns={"id":"xy"})[["xy","x","y","tokenId","type",]]
tiles

In [ ]:
events_land = pd.merge(events_land, tiles, how="left", left_on="asset.token_id", right_on="tokenId")

In [ ]:
for v in ["bid_amount","starting_price","ending_price","total_price"]:
    events_land[v] = events_land.apply(lambda row: float(row[v]) / 10**row["payment_token.decimals"] \
                                       if row[v] is not None else row[v], axis=1)

In [ ]:
cols = ["id","asset.token_id","created_date","transaction.timestamp","auction_type","event_type","bid_amount",
        "starting_price","ending_price","total_price",
        "payment_token.symbol","payment_token.usd_price",
        "asset.owner.address","from_account.address","to_account.address"]
events_land[cols]

In [ ]:
events_land[cols].event_type.value_counts()

In [ ]:
events_land["payment_token.symbol"].value_counts()

In [ ]:
tmp = events_land[events_land["event_type"] == "successful"][cols]
tmp.groupby("asset.owner.address")["asset.token_id"].nunique().sort_values()#.hist()